In [1]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import connect4
importlib.reload(connect4);
from connect4 import Connect4

In [2]:
#
# Hyper parameters
# 
alpha = 0.3
gamma = 0.9
epsilon = 0.3

In [3]:
#
# Create the model and optimizer
# 
model = nn.Sequential(
    nn.Linear(126, 252),
    nn.ReLU(),
    nn.Linear(252, 252),
    nn.ReLU(),
    nn.Linear(252, 7)
)
games = 0

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr = alpha)

In [5]:
#
# Load model from checkpoint
#
games = 2300000
cp = torch.load(f'connect4.nn');
model.load_state_dict(cp['model_state_dict']);
optimizer.load_state_dict(cp['optimizer_state_dict']);

In [5]:
@torch.no_grad()
def checkpoint(step):
    train = model.training
    if train: model.eval();
    print(f"{step}: checkpoint...")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'connect4-{step}.nn');

    dummy_input = Connect4().state
    torch.onnx.export(model, dummy_input, f"connect4-{step}.onnx");

    print(f"{step}: checkpoint saved.")
    if train: model.train();


In [6]:
#
# Validation
#
@torch.no_grad()
def validate(games):
    print('Validation...')
    xw, xd, xl, ow, od, ol  = connect4.validate(model, games)
    xtotal = xw + xd + xl
    ototal = ow + od + ol
    print(f"Cross: {100*xw/xtotal:.2f}% of {xtotal} ({xw}/{xd}/{xl})")
    print(f"Circle: {100*(ow+od)/ototal:.2f}% of {ototal} ({ow}/{od}/{ol})")

In [ ]:
#
# TRAINING
#
log_interval = 10000
validation_interval = 50000
validation_games = 20000
checkpoint_interval = 50000
losses = []
model.train()
for _ in range(2000000):
    env = Connect4()
    done = False
    games += 1
    loss = 0
    moves = 0

    while not done:
        q = model(env.state)
        targetq = q.detach().clone()

        e = random.uniform(0, 1)
#        if moves == 0:
#           e /= 2
        if e < epsilon:
            action = random.choice([a for a in range(7) if env.is_valid(a)])
        else:
            action = max([a for a in range(7) if env.is_valid(a)], key=lambda x: q[x])
        
        env.move(action)

        if env.winner != 0:
            targetq[action] = 1
            done = True
        elif env.full:
            targetq[action] = 0 if env.player == 2 else 0.25
            done = True
        else:
            model.eval()
            with torch.no_grad():
                next_q = model(env.state)
                next_max = -max([next_q[a] for a in range(9) if env.is_valid(a)])
                reward = -0.1 if env.player == 2 else 0
                targetq[action] = reward + gamma * next_max
            model.train()
            
        loss += F.mse_loss(q, targetq)
        moves += 1

    loss /= moves
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if games % log_interval == 0:
        print(f'{games}: average loss: {sum(losses)/len(losses)}')
        losses = []
    if games % validation_interval == 0:
        validate(validation_games)
    if games % checkpoint_interval == 0:
        checkpoint(games)
